1. 문서 내용 읽기
2. 문서 분할(쪼개기)
    - 문서를 분할하지 않으면,
        - 토큰 수 초과로 답변을 생성하지 못할 수 있음.
        - 문서 길이(input)가 길어 답변 생성에 시간 소요
3. 임베딩(수치화)
    - 벡터 데이터베이스에 저장
4. 질문이 있으면 벡터 데이터베이스에 유사도 검색
5. 유사도 검색으로 가져온 문서를 LLM에 질문과 같이 전달

In [ ]:
# 0. 패키지/라이브러리 설치
# %pip install -qU docx2txt langchain_community langchain-text-splitters langchain_chroma
# %pip install langchain-pinecone
# %pip install pinecone

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os
from dotenv import load_dotenv
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter


# 1. 문서 내용 읽고 분할 #######################################
loader = Docx2txtLoader('law_1.docx') # --> 인스턴스 생성

text_splitter = RecursiveCharacterTextSplitter( # --> 분할 설정
    chunk_size=1500,
    chunk_overlap=200,
)

document_list = loader.load_and_split(text_splitter=text_splitter)

# 2. 임베딩 -> 백터 데이터베이스에 저장  #########################
## 2.1 환경변수 읽어오기
load_dotenv()

## 2.2 임베딩 모델 지정
embedding = OpenAIEmbeddings(
    model='text-embedding-3-large'
)

## 3. Pinecone Vector DB
api_key = os.getenv('PINECONE_API_KEY')
pc = Pinecone(api_key=api_key)

database = PineconeVectorStore.from_documents(
    index_name='law-index',
    embedding=embedding,
    documents=document_list,    
)

# 4. 질문이 있으면, 벡터 데이터베이스에서 유사도 검색  #############
## 4.1 사용자 질문
query = '전세사기피해자 금융지원에 대해 설명해주세요.'

## 4.2 벡터 데이터베이스에서 유사도 검색
retrieved_docs = database.similarity_search(query=query, k=2)

# 5. 유사도 검색으로 가져온 문서를 LLM에 질문과 같이 전달 ###########
## 5.1 프롬프트 작성
prompt = '''
[identity]
- 당신은 전세사기피해 법률 전문가입니다.
- [context]를 참고하여 사용자의 질문에 답변해주세요.

[context]
{retrieved_docs}

Question: {query}
'''

## 5.2 프롬프트 변수에 값 설정(format 함수)
formatted_prompt = prompt.format(retrieved_docs=retrieved_docs, query=query)
 
## 5.3 LLM 모델 생성(ChatOpenAI 인스턴스 생성)
llm = ChatOpenAI()

## 5.4 LLM 모델에 질문과 검색된 문서를 보냄
ai_message = llm.invoke(formatted_prompt)
ai_message.content

c:\Users\2\miniconda3\envs\project\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


NameError: name 'database' is not defined